In [36]:
## working file

import pandas as pd
df = pd.DataFrame(index=range(8), columns=range(8))
df_whites = pd.DataFrame(index=range(8), columns=range(8))
df_blacks = pd.DataFrame(index=range(8), columns=range(8))
df_whites_movement = pd.DataFrame(index=range(8), columns=range(8))
df_blacks_movement = pd.DataFrame(index=range(8), columns=range(8))
df_pinned_by_whites = pd.DataFrame(index=range(8), columns=range(8))
df_pinned_by_blacks = pd.DataFrame(index=range(8), columns=range(8))

for row in range(0, 8):
    for column in range(0, 8):
        df.iloc[row, column] = 0
        df_whites.iloc[row, column] = 0
        df_whites_movement.iloc[row, column] = 0
        df_pinned_by_whites.iloc[row, column] = 0
        df_blacks.iloc[row, column] = 0
        df_blacks_movement.iloc[row, column] = 0
        df_pinned_by_blacks.iloc[row, column] = 0

## old development function which might be used later
def dfTo63():
    counter = 0
    for row in range(0, len(df.index)):
        for column in df.columns:
            df.iloc[row, column] = counter
            df_whites.iloc[row, column] = counter
            df_blacks.iloc[row, column] = counter
            counter += 1

## this function is an continuation of the locationEmpty function, but it is split up for cleanliness
def addMovement(row, column, is_white, previous_counter, added_counter):
    ## checking if the piece is white
    if is_white == True:
        if added_counter == 0:
            df_whites_movement.iloc[row, column] += 1
            df_pinned_by_whites.iloc[row, column] += 1
        elif added_counter == 1:
            if previous_counter == 0:
                df_whites_movement.iloc[row, column] += 1
            df_pinned_by_whites.iloc[row, column] += 1
        else:
            if previous_counter == 1:
                df_pinned_by_whites.iloc[row, column] += 1
    
    ## checking if the piece is black
    if is_white == False:
        if added_counter == 0:
            df_blacks_movement.iloc[row, column] += 1
            df_pinned_by_blacks.iloc[row, column] += 1
        elif added_counter == 1:
            if previous_counter == 0:
                df_blacks_movement.iloc[row, column] += 1
        else:
            if previous_counter == 1:
                df_pinned_by_blacks.iloc[row, column] += 1

## needs an new name for new functionality but is still an work in progress
def locationEmpty(row, column, is_white, counter):
    ## this function is called by a loop an is supplied with an counter, between 0 and 2. if the counter is 0 the piece has not encounter an other piece on its path. 
    ## if it already has encountered a singular opponent it recieves an 1. which means that it can not move any further but still pressures the piece behind.
    ## once it encounters an 2nd opponent or a friendly piece it gets a 2. as a result it cannot reach that spot under any circumstance by the next round
    
    #it passes this value on to the next function and once more but it adds the value for the spot it is checking right now, 0 for unoccupied, 1 for opponent and 2 for friendly  
    if  (df_whites.iloc[row, column] == 1 and is_white == True) or (df_blacks.iloc[row, column] == 1 and is_white == False):
        addMovement(row, column, is_white, counter, counter + 2)
        return 2 
    elif df.iloc[row, column] == 1:
        addMovement(row, column, is_white, counter, counter + 1)
        return 1
    else:
        addMovement(row, column, is_white, counter, counter + 0)
        return 0
    
    
def moveLeft(row, i, is_white, single_move=False ):
    counter = 0
    array = []
    for column in df.columns:
        if column < i:
            occupied = locationEmpty(row, (i - column) - 1, is_white, counter)
            counter = counter + occupied
            if counter > 2:
                counter = 2
            array.append(counter)
            if single_move == True:
                return array
    return array
def moveRight(row, i, is_white, single_move=False):
    counter = 0
    array = []
    for column in df.columns:
        if column > i:
            occupied = locationEmpty(row, column, is_white, counter)
            counter = counter + occupied
            if counter > 2:
                counter = 2
            array.append(counter)
            if single_move == True:
                return array
    return array
def moveUp(i, column, is_white, single_move=False):
    counter = 0
    array = []
    for row in range(0, len(df.index)):
        if row < i:
            occupied = locationEmpty((i - row) - 1, column, is_white, counter)
            counter = counter + occupied
            if counter > 2:
                counter = 2
            array.append(counter)
            if single_move == True:
                return array
    return array
def moveDown(i, column, is_white, single_move=False):
    counter = 0
    array = []
    for row in range(0, len(df.index)):
        if row > i:
            occupied = locationEmpty(row, column, is_white, counter)
            counter = counter + occupied
            if counter > 2:
                counter = 2
            array.append(counter)
            if single_move == True:
                return array
    return array
def moveNE(row, column, is_white, single_move=False):
    counter = 0
    array = []
    for i in range(1, 8):
        if row - i >= 0:
            if column + i < 8:
                occupied = locationEmpty(row - i, column + i, is_white, counter)
                counter = counter + occupied
                if counter > 2:
                    counter = 2
                array.append(counter)
                if single_move == True:
                    return array
    return array
def moveSE(row, column, is_white, single_move=False):
    counter = 0
    array = []
    for i in range(1, 8):
        if row + i < 8:
            if column + i < 8:
                occupied = locationEmpty(row + i, column + i, is_white, counter)
                counter = counter + occupied
                if counter > 2:
                    counter = 2
                array.append(counter)
                if single_move == True:
                    return array
    return array
def moveSW(row, column, is_white, single_move=False):
    counter = 0
    array = []
    for i in range(1, 8):
        if row + i < 8:
            if column - i >= 0:
                occupied = locationEmpty(row + i, column - i, is_white, counter)
                counter = counter + occupied
                if counter > 2:
                    counter = 2
                array.append(counter)
                if single_move == True:
                    return array
    return array
def moveNW(row, column, is_white, single_move=False):
    counter = 0
    array = []
    for i in range(1, 8):
        if row - i >= 0:
            if column - i >= 0:
                occupied = locationEmpty(row - i, column - i, is_white, counter)
                counter = counter + occupied
                if counter > 2:
                    counter = 2
                array.append(counter)
                if single_move == True:
                    return array
    return array
    
class Piece():
    occupied_locations = {}
    opponent_locations = {}
    piece_types = {"pawn": 1, "rook": 1, "knight": 1,
                   "bischop": 1, "queen": 1, "king": 1}

    def __init__(self, name, location, is_white):
        self.name = name
        self.location = location
        self.is_white = is_white
        self.has_moved = False
        Piece.occupied_locations[name] = location

    def hasMoved(self):
        return self.has_moved

    def moved(self):
        self.has_moved = True

    def isWhite(self):
        return self.is_white
    
    def showLocation(self):
        return self.location
    
    white_enpassant = False
    black_enpassant = False
    
    def setWhiteEnpassant(self, location):
        self.white_enpassant = location
        
    def removeWhiteEnpassant(self):
        self.white_enpassant = False
        
    def setBlackEnpassant(self, location):
        self.black_enpassant = location
        
    def removeBlackEnpassant(self):
        self.black_enpassant = False
    
class PawnWhite(Piece):
    possible_moves = []
    
class Rook(Piece):
    moves_up = {}
    moves_down = {}
    moves_left = {}
    moves_right = {}
    
    def showMoves(self):
        return [self.moves_up, self.moves_down, self.moves_left, self.moves_right]
    
    def setMoves(self):
        self.moves_up = moveUp(self.location[0], self.location[1], self.is_white)
        self.moves_down = moveDown(self.location[0], self.location[1], self.is_white)
        self.moves_left = moveLeft(self.location[0], self.location[1], self.is_white)
        self.moves_right = moveRight(self.location[0], self.location[1], self.is_white)
        
    def pieceMoveUp(self):
        self.moves_up = moveUp(self.location[0], self.location[1], self.is_white)

    def pieceMoveDown(self):
        self.moves_down = moveDown(self.location[0], self.location[1], self.is_white)
    
    def pieceMoveLeft(self):
        self.moves_left = moveLeft(self.location[0], self.location[1], self.is_white)
    
    def pieceMoveRight(self):
        self.moves_right = moveRight(self.location[0], self.location[1], self.is_white)
    
    
    
    
    
    
class Knight(Piece):
    possible_moves = []
class Bischop(Piece):
    possible_moves = []
class Queen(Piece):
    possible_moves = []
class King(Piece):
    possible_moves = []

In [37]:
df.iloc[3,3] = 1
df.iloc[2,3] = 1
df.iloc[6,3] = 1
df_whites.iloc[6,3] = 1

In [38]:
df_whites

,0,1,2,3,4,5,6,7
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0
6,0,0,0,1,0,0,0,0
7,0,0,0,0,0,0,0,0


In [39]:
moveUp(6,3, True)
print(df_whites_movement)
print()
print(df_pinned_by_whites)

   0  1  2  3  4  5  6  7
0  0  0  0  0  0  0  0  0
1  0  0  0  0  0  0  0  0
2  0  0  0  0  0  0  0  0
3  0  0  0  1  0  0  0  0
4  0  0  0  1  0  0  0  0
5  0  0  0  1  0  0  0  0
6  0  0  0  0  0  0  0  0
7  0  0  0  0  0  0  0  0

   0  1  2  3  4  5  6  7
0  0  0  0  0  0  0  0  0
1  0  0  0  0  0  0  0  0
2  0  0  0  1  0  0  0  0
3  0  0  0  1  0  0  0  0
4  0  0  0  1  0  0  0  0
5  0  0  0  1  0  0  0  0
6  0  0  0  0  0  0  0  0
7  0  0  0  0  0  0  0  0
